# Pydantic

Pydantic to biblioteka służąca do tworzenia i walidacji modeli struktur danych czyli schematów danych o strukturze typu JSON.

Przykładowe **zastosowania** modeli pydantica:
- definiowanie request body w REST API
- definiowanie formatu formatu JSONa według którego odpowiedź ma wygenerować LLM
- walidacja poprawności formatu danych JSON
- zamiana danych w postaci słownika na instancję klasy modelu (możemy wtedy np. zdefiniować dla takich obiektów odpowiednie metody)

## `BaseModel` i modele danych

In [1]:
from pydantic import BaseModel

In [2]:
class User(BaseModel):
    username: str
    password: str
    is_admin: bool

In [3]:
class Task(BaseModel):
    description: str
    priority: int
    is_completed: bool
    assigned_user: User

## Walidacja JSONa

Do walidacji JSONa względem modelu używamy metody `model_validate()`.

**Poprawna walidacja**

W przypadku kiedy słownik pasuje do modelu, `model_validate()` zwraca instancję tego modelu.

In [13]:
user = {"username": "John Doe", "password": "my_password", "is_admin": True}

task = {"description": "Task description", "priority": 2, "is_completed": False, "assigned_user": user}

In [14]:
user

{'username': 'John Doe', 'password': 'my_password', 'is_admin': True}

In [15]:
User.model_validate(user)

User(username='John Doe', password='my_password', is_admin=True)

In [16]:
Task.model_validate(task)

Task(description='Task description', priority=2, is_completed=False, assigned_user=User(username='John Doe', password='my_password', is_admin=True))

---

Poniżej `priority` jest zapisane jako `str`. Ponieważ jednak da się to zrzutować na `int`, to walidacja również przebiega poprawnie.

In [17]:
user = {"username": "John Doe", "password": "my_password", "is_admin": True}
task = {"description": "Task description", "priority": "2", "is_completed": False, "assigned_user": user}

In [18]:
Task.model_validate(task)

Task(description='Task description', priority=2, is_completed=False, assigned_user=User(username='John Doe', password='my_password', is_admin=True))

**Niepoprawna walidacja**

W przypadku błędu walidacji dostajemy błąd `ValidationError`.

In [19]:
user = {"username": "John Doe", "password": "my_password", "is_admin": True}
task = {"description": "Task description", "priority": "invalid type", "is_completed": False, "assigned_user": user}

In [20]:
Task.model_validate(task)

ValidationError: 1 validation error for Task
priority
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='invalid type', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/int_parsing

## Rzutowanie słownika na obiekt

Rzutowanie słownika na obiekt pozwala nie tylko przechowywać informacje w postaci tego obiektu zamiast słownika ale również korzystać ze wszystkich metod odpowiadającej mu klasy.

In [21]:
user

{'username': 'John Doe', 'password': 'my_password', 'is_admin': True}

In [23]:
User(**user)

User(username='John Doe', password='my_password', is_admin=True)

In [24]:
class User(BaseModel):
    username: str
    password: str
    is_admin: bool

    def change_password(self, new_password):
        if isinstance(new_password, str):
            self.password = new_password
        else:
            raise TypeError("Invalid type for new password")

In [26]:
user_obj = User(**user)
user_obj

User(username='John Doe', password='my_password', is_admin=True)

In [27]:
user_obj.change_password("new_password")
user_obj

User(username='John Doe', password='new_password', is_admin=True)

In [28]:
user_obj.change_password(True)

TypeError: Invalid type for new password

> **ZADANIE**

Stwórz klasę `Task`, która będzie miała następujące pola (atrybuty). Dobierz dla nich odpowiednie typy.
   - `description`
   - `assigned_user`
   - `due_date`
   - `comments`

Klasa ta powinna również posiadać metodę `modify_description()`, której zadaniem będzie modyfikacja opisu zadania. Jeśli podano wartość typu innego niż `str` należy zwrócić odpowiedni błąd.

Następnie na podstawie słownika z danymi o zadaniu dokonaj walidacji i zamiany słownika na obiekt.

In [29]:
from datetime import date

In [98]:
class Task(BaseModel):
    description: str = Field(alias="Opis")
    # assigned_user: User
    due_date: date
    comments: list[str]

    def modify_description(self, value: str):
        if not isinstance(value, str):
            raise TypeError("Description must be a string")
        self.description = value

In [99]:
task_dict = {
    "Opis": "Opis", "assigned_user": user, "due_date": date.today(), "comments": []
}

In [100]:
task_obj = Task(**task_dict)
task_obj

Task(description='Opis', due_date=datetime.date(2025, 9, 22), comments=[])

In [36]:
task_obj.modify_description("Nowy opis")

In [37]:
task_obj

Task(description='Nowy opis', assigned_user=User(username='John Doe', password='my_password', is_admin=True), due_date=datetime.date(2025, 9, 22), comments=[])

---
---
---

## Zaawansowane elementy definiowania modeli

**Dopuszczanie różnych typów**

In [38]:
class Task(BaseModel):
    description: str
    priority: int | str
    is_completed: bool | None

In [43]:
Task(description="task description", priority=3, is_completed=None)

Task(description='task description', priority=3, is_completed=None)

---

In [44]:
class Task(BaseModel):
    description: str
    priority: float  # | int
    is_completed: bool | None

In [45]:
Task(description="task description", priority=2, is_completed=False)

Task(description='task description', priority=2.0, is_completed=False)

**Typ `Literal`**

In [46]:
from typing import Literal

In [47]:
class Task(BaseModel):
    description: str
    priority: Literal["low", "medium", "high"]
    is_completed: bool

In [50]:
Task(description="task description", priority="medium", is_completed=True)

Task(description='task description', priority='medium', is_completed=True)

**Typy z ograniczeniami**

Constrained types

In [51]:
from pydantic import constr, conint, confloat

In [52]:
class Test(BaseModel):
    test_string: constr(min_length=5, max_length=10, pattern=r"^[a-zA-Z]+$")
    test_int: conint(gt=2, lt=8)
    test_float: confloat(ge=0.0, le=1.0)

In [67]:
test = Test(test_string="xyzab", test_int=4, test_float=0.5)
test

Test(test_string='xyzab', test_int=4, test_float=0.5)

**Wartości domyślne i opcjonalne**

In [68]:
from datetime import date

In [69]:
class User(BaseModel):
    username: str
    password: str
    is_admin: bool = False
    date_of_birth: date | None = None

In [70]:
User(username="John Doe", password="password")

User(username='John Doe', password='password', is_admin=False, date_of_birth=None)

**Field – metadane pola**

`Field` pozwala zdefiniować wiele rzeczy w jednym miejscu, m.in. wartość domyślną, walidację wartości, alias pola czy jego opis.

In [71]:
from pydantic import Field
from datetime import datetime

In [90]:
class User(BaseModel):
    username: str = Field(alias=["login"])
    password: str = Field(min_length=8)
    is_admin: bool = Field(False, description="Whether or not the user is an admin")
    creation_timestamp: date = Field(default_factory=datetime.now)

In [92]:
User(login="my_username", password="my_password")

User(username='my_username', password='my_password', is_admin=False, creation_timestamp=datetime.datetime(2025, 9, 22, 15, 53, 16, 155500))

**`field_validator` i `model_validator` – customowa walidacja**

Jeśli chcemy zastosować bardziej zaawansowaną walidację modelu możemy użyć metod z dekoratorami `field_validator` lub `model_validator`.

`field_validator` - służy do walidacji wartości w pojedynczym polu

`model_validator` - służy do walidacji zależności między polami modelu

In [75]:
from pydantic import field_validator, model_validator

In [82]:
class User(BaseModel):
    username: str = Field(alias="login")
    password: str = Field(min_length=8)
    is_admin: bool = Field(False, description="Whether or not the user is an admin")
    creation_timestamp: date = Field(default_factory=datetime.now)

    @field_validator("password")
    def check_password_not_contains_admin(cls, value):
        if "admin" in value.lower():
            raise ValueError("Password should not contain the word 'admin'")
        return value

    @model_validator(mode="after")
    def validate_password_length_for_admins(self):
        min_admin_password_length = 12
        if self.is_admin and len(self.password) < min_admin_password_length:
            raise ValueError("Too short password for an admin")
        return self

In [84]:
User(login="user", password="user1234", is_admin=False)  # admin1234

AttributeError: 'dict' object has no attribute 'is_admin'

In [81]:
User(login="user", password="user1234", is_admin=True)  # user1234

ValidationError: 1 validation error for User
  Value error, Too short password for an admin [type=value_error, input_value={'login': 'user', 'passwo...1234', 'is_admin': True}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

> **ZADANIE**

Rozbuduj klasę `Task` z poprzedniego zadania w następujący sposób:
- w polu `assigned_user` zezwól na wartość typu `str` (username użytkownika)
- dodaj pole `category`, które będzie miało kilka określonych wartości dopuszczalnych
- ustaw maksymalną długość opisu na 30 znaków
- przyjmij pustą listę jako domyślną wartość pola `comments`
- dodaj opis wybranego pola korzystając z `Field`

Następnie stwórz instancję tej klasy.

In [101]:
class Task(BaseModel):
    description: str = Field(max_length=30)
    assigned_user: User | str
    category: Literal["bug", "feature", "task"] = Field(description="Task category")
    due_date: date
    comments: list[str] = []

In [108]:
Task(
    description="Opis zadania",
    assigned_user="username",
    category="bug",
    due_date=date.today()
)

Task(description='Opis zadania', assigned_user='username', category='bug', due_date=datetime.date(2025, 9, 22), comments=[])

## Pydantic + LLM

Poniższego kodu **nie należy** wykonywać (jest do tego potrzebny klucz API OpenAI). Obrazuje on jednak w jaki sposób możemy zastosować Pydantica w pracy z modelami językowymi.

In [ ]:
from openai import OpenAI

In [ ]:
client = OpenAI()

In [ ]:
class User(BaseModel):
    username: str = Field(description="Username of the user")
    password: str = Field(description="Password of the user")
    is_admin: bool = Field(description="Whether or not the user is an admin")

In [ ]:
response = client.responses.parse(
    model="gpt-4o",
    input=[
        {
            "role": "user",
            "content": "Generate a sample user for test purposes",
        },
    ],
    text_format=User,
)

In [ ]:
response.output[0].content[0].text

`'{"username":"test_user123","password":"secureP@ssw0rd!","is_admin":false}'`

---

In [ ]:
response.output_parsed

`User(username='test_user123', password='secureP@ssw0rd!', is_admin=False)`